Hi all, 
    my name is Jacopo Pierotti and this is my first attempt to analyze some data distribution.
In the following notebook, we will analyze the data distribution of the shooting from the police in the U.S.A. during the time period 2015 to mid 2020.
MOst of this work is based on the very clear notebook by Mrinal (https://www.kaggle.com/mrinaal007/police-shootouts) which I invite you to check.
My main extra points with respect to his work are the normalization of the people based on the U.S.A. demographic.

First of all, we import some useful libraries needed for our analyses

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px # data visualization tool
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Now we load our data and we take a look at the dataset.

In [ ]:
df = pd.read_csv('/kaggle/input/data-police-shootings/fatal-police-shootings-data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

As we can see, we have plenty of useful information but also unuseful one. Hence, we remove the unuseful coloum (id) from the dataset. Then, we transform the data in a more convenient format.

In [ ]:
df = df.drop('id', 1)
df['month'] = pd.to_datetime(df['date']).dt.month
df['year'] = pd.to_datetime(df['date']).dt.year

We check if the dataframe is missing some information (i.e. some entries are empty).<bf>
We compute the percentages of missing data for each coloumn.
As we will see, the coloumn with the most unavailable data doesn't have values for about 10% of its entries.
We decide that about 90% of the 5000+ rows of the dataframe is still an very good amount of data to consider, so we drop entries without complete information.   


In [ ]:
uncomplete_data = df.isna().sum()*100/df.shape[0]
print(uncomplete_data)
df.dropna(inplace=True)

Now we check how many different value each coloumn can take and, to ease the reading, we change the string values some entries.
<br> Trick for non-U.S. citizen not famliar with foreing geography (such as myself), the caridnality of the state coloumn is 51 because there are 50 state + washinton dc which does not belong to any state.

In [ ]:
cardinality ={}
for col in df.columns:
    cardinality[col] = df[col].nunique()
#print(cardinality)
d = {'A':'Asian','B':'Black','H':'Hispanic', 'N':'Native','O':'Other','W':'White'}
df['race'] = df['race'].replace(d)

Now we start actually analyzing some data. For starter, we display how many shooting -in absolute terms- happend over all the years. 

In [ ]:
yearly_shootouts = df['year'].value_counts()
yearly_shootouts = pd.DataFrame(yearly_shootouts)
yearly_shootouts= yearly_shootouts.reset_index()
yearly_shootouts=yearly_shootouts.rename(columns={'index':'year','year':'Shootouts'})
fig = px.bar(yearly_shootouts, y='Shootouts', x='year', barmode='group')
fig.show()

The trend seems to be decreasing (note that the data for 2020 is incomplete). To have a more accurate idea, we normalize the above figure with respect to the u.s.a. population for those years (data taken form wikipedia). <br>
*for the data analyst out there, how do you divide a column of a dataframe by a dictionary (where the keys of the dictionary are the row of the dataframe)?*

In [ ]:
#yearly_population = {'2015':320.7,'2016':323.1,'2017':325.1,'2018':327.2 ,'2019':328.2,'2020':331.0}

yearly_population_array = [320.7,323.1,325.1,327.2 ,328.2,331.0]
yearly_shootouts['Shootouts_normalized'] = yearly_shootouts['Shootouts'].div(yearly_population_array).multiply(yearly_population_array[-1])
fig = px.bar(yearly_shootouts, y='Shootouts_normalized', x='year', barmode='group')
fig.show()

The previous two barplot show the first main point of this analysys.<br>
# 1) Police shooting (in absolute and relative terms) have been decreasing in the past 6 years 
<br>
Note that this is just an observation based on 6 years, which might not be enough to extract a general trend. <br>
Now we look at other features, such as age distribution, usage of body cams, gender and etnicity.

In [ ]:
import plotly.figure_factory as ff
for etn in df['race'].unique():
    x = df[df['race']==etn]['age']
    hist_data = [x]
    group_labels = ['Age_'+str(etn)]
    fig = ff.create_distplot(hist_data, group_labels)
    fig.show()

As shown from the graph above, most victims are between their twenties and thirties, independently of their etnicity.
Now we analyze the usage of body cams.

In [ ]:
bc_shootout = df['body_camera']
print(bc_shootout.value_counts())
print(bc_shootout.value_counts()[0]/bc_shootout.value_counts()[1])
fig = px.histogram(bc_shootout,x='body_camera',color='body_camera')
fig.show()

The graph above lead to the following main consideration: <br>
# 2) Police officers are 7.3 times more likely to shoot when their body cameras are deactivated.<br>
One small note about this, our data don't tell us if all the police officers had the option to have and wear a body camera. If they didn't the previous point would not be correct.

<br>Hereafter, we analize the gender inequality in police shootings.

In [ ]:
for etn in df['race'].unique():
    gender_per_etn = df[df['race']==etn]['gender']
    fig = px.histogram(gender_per_etn,x='gender',color='gender',labels={'gender':'gender_'+str(etn)})
    fig.show()   
gender_shootout = df['gender']
print(gender_shootout.value_counts())
print(gender_shootout.value_counts()[0]/gender_shootout.value_counts()[1])

The previous histograms show us a huge gap between the genders (given the structur of the dataframe, just male and female have been considered).
# 3) Males are 21.33 times more likely to get shot then females.<br>
Next, we confront shooting by etnicity, first in absolte terms, then in relative terms.
Again, the missing data are taken from wikipedia and refers to the year 2017.

In [ ]:
etn_shootout = df['race']
print(etn_shootout.value_counts())
fig = px.histogram(etn_shootout,x='race',color='race')
fig.show()

It's clear that, in absolute term, white people are shot more than anybody else. But white people constitute around 61.5% of the overall population. Let's normalize and see what happen.
<br>
Again, the missing data are taken from wikipedia and refers to the year 2017, due to some small approxiamtion errors (most likely), the sum of the percentage adds up to 100.2, but this small error won't stir the overall trends.

In [ ]:
etn_population_array = [61.5,12.7,17.3, 5.3,0.9,2.5] # unelegant - to correct later
etn_shootouts_norm = pd.DataFrame()
etn_shootouts_norm['etn_shootouts_norm'] = df['race'].value_counts().divide(etn_population_array)

print(etn_shootouts_norm.head())
fig = px.bar(etn_shootouts_norm,y='etn_shootouts_norm',color='etn_shootouts_norm', barmode='group')
fig.show()

These results are probably the most interesting of the all discussion.
# 4) The trends clearly shows that black and native people are more likely to be shot, followed by hispanic and white people while asian and other nationality are the least likely.

<br> Among the many other things we could still analyze, we decide to focus on the shooting against disarmed and not fleeing people who also don't show any sign of mental illness per etnicity. In some sense, these shooting can be seen as the most unjustified ones. To make a fair comparison, we normalize these shooting with respect to the total count of shooting per etnicity.

In [ ]:
innocent_people = df[(df.signs_of_mental_illness==False) & (df.armed =='unarmed') & (df.flee=='Not fleeing')]
lista= pd.DataFrame()
lista['count'] = innocent_people['race'].value_counts().divide(df['race'].value_counts())

fig = px.bar(lista, y='count', barmode='group')
fig.show()

We can see that:<br>
# 5) the ratio of very unjustified shooting over total shooting per etnicity is quite constant among all the etnicities.<br>
(Native and Other present different result because of the small amount of sample) 

This was my analisys on the dataset '/kaggle/input/data-police-shootings/fatal-police-shootings-data.csv' and it is also my first real data analisys. Don't be shy, constructive feedback and comments are very welcome :)